>Build a database of restaurants, bars and various points of interest (POIs) in the area of your choice

> Areas: 
> * Tempe, AZ, USA
> * Hyderabad, AP, India

# Startup code

In [1]:
import requests
import json
import sqlite3
from pathlib import Path
import os

import numpy as np
import pandas as pd

import configs as cfg

from IPython.display import JSON

In [2]:
near= "33.4363964,-111.9453846" #lat,long
radius = '20000'
fs_category_id = '4d4b7105d754a06374d81259,4bf58dd8d48988d116941735'
limit=50
# near = 'Tempe,AZ'

# Foursquare API

In [3]:
foursquare_id = cfg.API_KEYS["four_square"]["client_id"]
foursquare_secret = cfg.API_KEYS["four_square"]["client_secret"]

In [4]:
fs_root_url = 'https://api.foursquare.com/v2/'

venue_search = 'venues/search'
categories = 'venues/categories'
venue_details = 'venues/'

In [5]:
def get_response(root_url,endpoint,params=None,headers=None):
    url = root_url+endpoint
    try:
        response = requests.get(url,params=params,headers=headers)
        if response.status_code != 200:
            print("Error in fetching data")
        return response
    except Exception as e:
        print(f"Error in making the call to provided resource. Is the url correct? {url}")
        print(e)

## Get Categories

In [6]:
# Get venue categories

params={
    'client_id':foursquare_id,
    'client_secret':foursquare_secret,
    'v':'20210630'
}

category_response = get_response(fs_root_url,categories,params)

JSON(category_response.json())

<IPython.core.display.JSON object>

In [7]:
df_four_square_categories = pd.json_normalize(category_response.json(),record_path=['response','categories','categories'],
                                              meta=[['response','categories','name'],['response','categories','id']])
df_four_square_categories = df_four_square_categories.rename(columns={
    'id':'Category Id',
    'name':'Category Name',
    'response.categories.name': 'Parent Category Name',
    'response.categories.id':'Parent Category Id'
})

df_four_square_categories = df_four_square_categories.reindex(columns=['Parent Category Id','Parent Category Name','Category Id','Category Name'])
df_four_square_categories.head()

,Parent Category Id,Parent Category Name,Category Id,Category Name
0,4d4b7104d754a06370d81259,Arts & Entertainment,56aa371be4b08b9a8d5734db,Amphitheater
1,4d4b7104d754a06370d81259,Arts & Entertainment,4fceea171983d5d06c3e9823,Aquarium
2,4d4b7104d754a06370d81259,Arts & Entertainment,4bf58dd8d48988d1e1931735,Arcade
3,4d4b7104d754a06370d81259,Arts & Entertainment,4bf58dd8d48988d1e2931735,Art Gallery
4,4d4b7104d754a06370d81259,Arts & Entertainment,4bf58dd8d48988d1e4931735,Bowling Alley


In [8]:
df_four_square_categories.to_csv(r'./data/four_square_categories.csv')

In [9]:
df_parent_categories_fs = df_four_square_categories[['Parent Category Id','Parent Category Name']]
df_child_categories_fs = df_four_square_categories[['Category Id','Category Name','Parent Category Id']]

In [10]:
df_parent_categories_fs = df_parent_categories_fs.value_counts()
df_parent_categories_fs = df_parent_categories_fs.reset_index()
df_parent_categories_fs = df_parent_categories_fs.drop(columns=[0])
df_parent_categories_fs

,Parent Category Id,Parent Category Name
0,4d4b7105d754a06378d81259,Shop & Service
1,4d4b7105d754a06374d81259,Food
2,4d4b7105d754a06377d81259,Outdoors & Recreation
3,4d4b7105d754a06375d81259,Professional & Other Places
4,4d4b7104d754a06370d81259,Arts & Entertainment
5,4d4b7105d754a06379d81259,Travel & Transport
6,4d4b7105d754a06372d81259,College & University
7,4d4b7105d754a06373d81259,Event
8,4d4b7105d754a06376d81259,Nightlife Spot
9,4e67e38e036454776db1fb3a,Residence


In [11]:
df_child_categories_fs.head()

,Category Id,Category Name,Parent Category Id
0,56aa371be4b08b9a8d5734db,Amphitheater,4d4b7104d754a06370d81259
1,4fceea171983d5d06c3e9823,Aquarium,4d4b7104d754a06370d81259
2,4bf58dd8d48988d1e1931735,Arcade,4d4b7104d754a06370d81259
3,4bf58dd8d48988d1e2931735,Art Gallery,4d4b7104d754a06370d81259
4,4bf58dd8d48988d1e4931735,Bowling Alley,4d4b7104d754a06370d81259


### Save categories into SQL database

In [12]:
con = sqlite3.connect('four_square_sample.db')

In [13]:
cur = con.cursor()

In [14]:
#split this table into maincategories and subcategories
#categories: category id pk, category name
#sub category: sub-category name, sub-category id pk, category id fk

In [15]:
df_parent_categories_fs = df_parent_categories_fs.rename(columns={'Parent Category Id':'parent_category_id','Parent Category Name':'parent_category_name'})
df_parent_categories_fs.to_sql(name='Parents_Category',con=con,if_exists='replace')

In [16]:
# cur.execute("""
# SELECT * FROM Parents_Category
# """).fetchall()

In [17]:
df_child_categories_fs = df_child_categories_fs.rename(columns={'Parent Category Id':'parent_category_id','Category Name':'category_name','Category Id':'category_id'})
df_child_categories_fs.to_sql(name='category',con=con,if_exists='replace')

In [18]:
# cur.execute("""
# SELECT * FROM category
# """).fetchall()

In [19]:
con.commit()
con.close()

## Venue Search

In [20]:
params={
    'client_id':foursquare_id,
    'client_secret':foursquare_secret,
    'v':'20210630',
    'radius':radius,
    'categoryId':fs_category_id,
    'near':near,
    'limit':limit
}
response = get_response(fs_root_url,venue_search,params=params)

In [21]:
JSON(response.json())

<IPython.core.display.JSON object>

In [22]:
df_foursquare = pd.json_normalize(response.json(),record_path=['response','venues','categories'],
                                  meta=[['response','venues','name'],['response','venues','location','lat'],['response','venues','location','lng'],
                                       ['response','venues','location','postalCode'],['response','venues','location','city'],
                                       ['response','venues','location','state'],['response','venues','location','country'],
                                       ['response','venues','id']],errors='ignore')
df_foursquare = df_foursquare.rename(columns={
    'name':'category name',
    'id' : 'category id',
   'response.venues.name':'Venue Name',
    'response.venues.id':'Venue id',
    'response.venues.location.lat':'Latitude',
    'response.venues.location.lng':'Longitude',
    'response.venues.location.postalCode':'Postal Code',
    'response.venues.location.city':'City',
    'response.venues.location.state':'State',
    'response.venues.location.country' : 'Country'
})
df_foursquare = df_foursquare.drop(columns=['pluralName','shortName','primary','icon.prefix','icon.suffix'])
df_foursquare = df_foursquare.reindex(columns=['Venue id', 'Venue Name','Latitude','Longitude','City','State','Country','Postal Code','category name','category id'])
df_foursquare = df_foursquare.astype({'Latitude':'float','Longitude':'float','Postal Code':'float'})
df_foursquare.tail()

,Venue id,Venue Name,Latitude,Longitude,City,State,Country,Postal Code,category name,category id
45,527b2c5811d242a17c47ce10,Four Peaks Brewing Company,33.436992,-112.001882,Phoenix,AZ,United States,NaN,Brewery,50327c8591d4c4b30a586d5d
46,5d68dd710181700008037d80,Blanco Block 23,33.447281,-112.071548,Phoenix,AZ,United States,85004.0,Restaurant,4bf58dd8d48988d1c4941735
47,5c3fc71716fa04002c961e01,SanTan Brewing Company,33.434263,-112.010239,Phoenix,AZ,United States,85034.0,Restaurant,4bf58dd8d48988d1c4941735
48,413ba880f964a520f81b1fe3,Priceless Prime Time Bar & Grill,33.350508,-111.877551,Chandler,AZ,United States,85224.0,Bar,4bf58dd8d48988d116941735
49,5c2c277ffb8e59002c448b0e,Starbucks,33.422132,-111.936834,Tempe,AZ,United States,85281.0,Coffee Shop,4bf58dd8d48988d1e0931735


In [23]:
list(zip(df_foursquare.columns,df_foursquare.dtypes))

[('Venue id', dtype('O')),
 ('Venue Name', dtype('O')),
 ('Latitude', dtype('float64')),
 ('Longitude', dtype('float64')),
 ('City', dtype('O')),
 ('State', dtype('O')),
 ('Country', dtype('O')),
 ('Postal Code', dtype('float64')),
 ('category name', dtype('O')),
 ('category id', dtype('O'))]

In [24]:
# Let's split our table into 3
# location: lat pk,long pk,city,state,country,postalcode
# categories: category name , category id pk
# venue: venue id pk, venue name, lat fk, lon fk, cat fk

In [25]:
df_location_fs = df_foursquare[['Latitude','Longitude','City','State','Country','Postal Code']]
df_venue_fs = df_foursquare[['Venue id','Venue Name','Latitude','Longitude','category id']]

df_location_fs = df_location_fs.rename(columns = {
    'Postal Code':'postal_code'
})
df_venue_fs = df_venue_fs.rename(columns =
{
    'Venue id':'venue_id',
    'Venue Name':'venue_name',
    'category id':'category_id'
}
)

df_location_fs.head()

,Latitude,Longitude,City,State,Country,postal_code
0,33.421663,-111.940889,Tempe,AZ,United States,85281.0
1,33.429761,-111.908645,Tempe,AZ,United States,85281.0
2,33.507151,-112.039467,Phoenix,AZ,United States,85016.0
3,33.581418,-112.127408,Phoenix,AZ,United States,85029.0
4,33.360837,-111.790051,Gilbert,AZ,United States,85234.0


In [26]:
con = sqlite3.connect('four_square_sample.db')

df_location_fs.to_sql(name='location',con=con,if_exists='replace')
df_venue_fs.to_sql(name='venues',con=con,if_exists='replace')

In [27]:
# cur = con.cursor()
# cur.execute("""
# SELECT * FROM venues;
# """).fetchall()

In [28]:
con.commit()
con.close()

### Ratings for each venue

In [29]:
df_venue_fs["venue_name"]

0                           Chick-fil-A
1                            Portillo's
2                       In-N-Out Burger
3                             Starbucks
4                  Flancer's Restaurant
5                             Starbucks
6                     Lone Butte Casino
7           Songbird Coffee & Tea House
8                     Cartel Coffee Lab
9            Four Peaks Brewing Company
10                            Starbucks
11                     The Sleepy Whale
12                            Starbucks
13                         Persian Room
14                       Sunrise Donuts
15                    Fry's Fuel Center
16                           McDonald's
17                            Starbucks
18                            The Henry
19                 Provision Coffee Bar
20                         Press Coffee
21               Chipotle Mexican Grill
22       Raising Cane's Chicken Fingers
23                          Gallagher's
24                            Starbucks


In [30]:
params={
    'client_id':foursquare_id,
    'client_secret':foursquare_secret,
    'v':'20210630'
}
ratings_fs_response = get_response(fs_root_url,venue_details+"58984e6aaf5c1416e0deedc9",params=params)

Error in fetching data


In [31]:
JSON(ratings_fs_response.json())

<IPython.core.display.JSON object>

In [32]:
#pass the venue id
def save_json(folder_path,venue_id):
    file_to_open = folder_path / f"{venue_id}.json"
    if not os.path.isfile(file_to_open): # if file does not exist
        with open(file_to_open, 'w') as f:
            params={
                'client_id':foursquare_id,
                'client_secret':foursquare_secret,
                'v':'20210630'
            }
            ratings_fs_response = get_response(fs_root_url,venue_details+venue_id,params=params)
            json.dump(ratings_fs_response.json(), f)

In [33]:
folder_path = Path("./four_square_ratings/")
if not os.path.exists(folder_path):
  os.mkdir(folder_path)
for venue in df_venue_fs["venue_id"]: 
    save_json(folder_path,str(venue))

Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data
Error in fetching data


In [34]:
def get_df_ratings(folder_path,venues):
#     pathlist = Path(folder_location).glob('*.json')
    rows = []
    for venue in venues:
        remove_file = False
        file_to_open = folder_path / f"{venue}.json"
        if os.path.exists(file_to_open):
            with open(file_to_open, 'r') as f:
                if os.path.getsize(file_to_open)!=0:
                    json_data = json.load(f)
                    if 'error' not in json_data and json_data["meta"]["code"]==200:
                        data = pd.json_normalize(json_data,record_path=['response','venue','categories'],
                                         meta=[['response','venue','id'],['response','venue','name'],
                                              ['response','venue','price','currency'],
                                              ['response','venue','likes','count'],['response','venue','rating'],
                                               ['response','venue','ratingSignals']
                                              ],errors='ignore')
                        data = data.drop(columns=[
                            'id','name','pluralName','shortName','icon.prefix','icon.suffix','primary'
                        ])
                        data = data.rename(columns={
                            'response.venue.id':'venue_id',
                            'response.venue.name':'venue_name',
                            'response.venue.price.currency':'price',
                            'response.venue.likes.count':'likes',
                            'response.venue.rating':'rating',
                            'response.venue.ratingSignals':'number_of_ratings',   
                        })
                        rows.append(data)
                    else:
                        print("ratings for this venue: {} not stored".format(venue))
                        remove_file=True
                else:
                    print("empty file: {}".format(file_to_open))
                    remove_file=True

            if remove_file==True:
                try:
                    os.remove(file_to_open)
                except Exception as e:
                    print(f"could not remove the file {e}")
        
    return pd.concat(rows)

In [35]:
venues = list(df_venue_fs["venue_id"])
df = get_df_ratings(folder_path,venues)

ratings for this venue: 4a593f65f964a52019b91fe3 not stored
ratings for this venue: 549c3af4498e520a4e173508 not stored
ratings for this venue: 529fbf7111d24bc7b8012e40 not stored
ratings for this venue: 5a4562367dc9e11be21e7fc1 not stored
ratings for this venue: 4a9c85b4f964a520753720e3 not stored
ratings for this venue: 4fe3c4bde4b0ec765db13204 not stored
ratings for this venue: 4c9460786b35a1437d321bdc not stored
ratings for this venue: 5c38d4766a5950002c0c91ab not stored
ratings for this venue: 609d273d426b352cc7190d63 not stored
ratings for this venue: 5e57035cb50c0100088d4a0c not stored
ratings for this venue: 413ba880f964a520351c1fe3 not stored
ratings for this venue: 500ec0a8f2e77036666940ad not stored
ratings for this venue: 50907b48498e8c94e352caf3 not stored
ratings for this venue: 5096b00be4b0b538e61eed46 not stored
ratings for this venue: 51ed7cbb498ec6594f87bf54 not stored
ratings for this venue: 527b2c5811d242a17c47ce10 not stored


In [36]:
df_ratings = df.reset_index(drop=True)
df_ratings = df_ratings.drop_duplicates()
df_ratings.sort_values(by='venue_id',ascending=False)

,venue_id,venue_name,price,likes,rating,number_of_ratings
42,5fac65ea1e934d1bd1169ac3,Monroe's Hot Chicken,$,3,NaN,NaN
43,5db759724e10f90008b97997,El Paisano Restaurant at the Rise,$,0,NaN,NaN
47,5d68dd710181700008037d80,Blanco Block 23,$,0,NaN,NaN
17,5ccb808aa795bd002c1db35f,The Sleepy Whale,NaN,18,NaN,NaN
48,5c3fc71716fa04002c961e01,SanTan Brewing Company,$,19,7.9,12
50,5c2c277ffb8e59002c448b0e,Starbucks,$,14,7.3,13
40,5bf0d1793c858d003949a480,Rancho La Candelaria,$,2,NaN,NaN
26,59753b12e075506cb4620572,Press Coffee,$,19,8.6,12
25,5847462126a953661ff93230,Starbucks,$,14,8.7,12
4,581b8af5195c4b5a43340207,In-N-Out Burger,$,34,9.0,53


In [37]:
df_venue_fs.sort_values(by='venue_id',ascending=False)

,venue_id,venue_name,Latitude,Longitude,category_id
36,609d273d426b352cc7190d63,Starbucks,33.498924,-111.917980,4bf58dd8d48988d1e0931735
35,5fac65ea1e934d1bd1169ac3,Monroe's Hot Chicken,33.434135,-111.927130,4d4ae6fc7a7b7dea34424761
38,5e57035cb50c0100088d4a0c,PHX Beer Co - Sky Harbor Airport,33.437414,-112.008876,56aa371ce4b08b9a8d57356c
37,5db759724e10f90008b97997,El Paisano Restaurant at the Rise,33.414505,-111.924302,4bf58dd8d48988d1c4941735
46,5d68dd710181700008037d80,Blanco Block 23,33.447281,-112.071548,4bf58dd8d48988d1c4941735
11,5ccb808aa795bd002c1db35f,The Sleepy Whale,33.299460,-111.841896,56aa371ce4b08b9a8d57356c
47,5c3fc71716fa04002c961e01,SanTan Brewing Company,33.434263,-112.010239,4bf58dd8d48988d1c4941735
33,5c38d4766a5950002c0c91ab,Starbucks,33.435633,-112.008431,4bf58dd8d48988d1e0931735
49,5c2c277ffb8e59002c448b0e,Starbucks,33.422132,-111.936834,4bf58dd8d48988d1e0931735
34,5bf0d1793c858d003949a480,Rancho La Candelaria,33.381084,-112.129163,4bf58dd8d48988d1cb941735


In [38]:
df_ratings = df_ratings.drop(columns=['venue_name'])
df_venues_with_ratings = pd.merge(df_ratings,df_venue_fs,on='venue_id')
df_venues_with_ratings.shape

(34, 9)

In [39]:
con = sqlite3.connect('four_square_sample.db')
df_venues_with_ratings.to_sql(name='venue_with_ratings',con=con,if_exists='replace')
con.commit()
con.close()

In [40]:
df_venues_with_ratings["rating"] = ((df_venues_with_ratings["rating"]-1)/(9))*5

In [41]:
df_venue_fs.to_csv('fs_ratings.csv')
df_venues_with_ratings.to_csv('fs_ratings.csv')
df_location_fs.to_csv("fs_location.csv")
df_child_categories_fs.to_csv("fs_child_categories.csv")
df_parent_categories_fs.to_csv("fs_parent_categories.csv")

# Yelp API

In [42]:
yelp_id = cfg.API_KEYS["yelp"]["client_id"]
yelp_key = cfg.API_KEYS["yelp"]["key"]

In [43]:
yelp_root_url = 'https://api.yelp.com/v3/'

headers = {
    'Authorization':f'BEARER {yelp_key}'
}

venue_search = 'businesses/search'
categories = 'categories'

## Yelp Categories

In [44]:
yelp_categories = get_response(yelp_root_url,categories,headers=headers)

In [45]:
JSON(yelp_categories.json())

<IPython.core.display.JSON object>

In [46]:
df_categories = pd.json_normalize(yelp_categories.json(),record_path=['categories','parent_aliases'],meta=[['categories','title']],errors='ignore')
df_categories = df_categories.rename(columns={
    0:"parent_category",
    "categories.title":'category'
})
df_categories.head()

,parent_category,category
0,localservices,3D Printing
1,italian,Abruzzese
2,bars,Absinthe Bars
3,food,Acai Bowls
4,fashion,Accessories


In [47]:
df_categories[df_categories.category.str.contains("pubs",case=False)]

,parent_category,category
195,breweries,Brewpubs
574,restaurants,Gastropubs
1149,bars,Pubs


### Save categories to SQL

In [48]:
con = sqlite3.connect("yelp_database.db")

In [49]:
df_categories.to_sql(name="categories",con=con,if_exists='replace')

In [50]:
# cur = con.cursor()
# cur.execute(
# """
# SELECT * FROM categories
# """).fetchall()

In [51]:
con.commit()
con.close()

## Venue Search

In [52]:
categories = 'restaurants,bars'
params = {
    'location':near,
    'radius':radius,
    'categories':categories,
    'limit':limit
}

yelp_response = get_response(yelp_root_url,venue_search,params,headers)

In [53]:
JSON(yelp_response.json())

<IPython.core.display.JSON object>

In [54]:
df_yelp = pd.json_normalize(yelp_response.json(),record_path=['businesses','categories'],
                           meta=[['businesses','id'],['businesses','name'],['businesses','rating'],['businesses','price'],['businesses','review_count'],
                                 ['businesses','location','address1'],['businesses','location','address2'],['businesses','location','address3'],
                                ['businesses','location','city'],['businesses','location','zip_code'],['businesses','location','country'],
                                ['businesses','location','state'],
                                ['businesses','coordinates','latitude'],['businesses','coordinates','longitude']])
df_yelp["address"] = df_yelp["businesses.location.address1"]+df_yelp["businesses.location.address2"]+df_yelp["businesses.location.address3"]

df_yelp = df_yelp.rename(columns={
    'businesses.location.state':'state',
    'businesses.location.country':'country',
    'businesses.location.zip_code':'zip_code',
    'businesses.location.city':'city',
    'businesses.id':'business_id',
    'businesses.name':'business_name',
    'alias':'category_alias',
    'title':'category',
    'businesses.coordinates.latitude':'lat',
    'businesses.coordinates.longitude':'lng',
    'businesses.rating':'rating',
    'businesses.price':'price',
    'businesses.review_count':'review_count'
})

df_yelp = df_yelp.drop(columns=[
    'businesses.location.address1',
    'businesses.location.address2',
    'businesses.location.address3'
])

df_yelp.head()

,category_alias,category,business_id,business_name,rating,price,review_count,city,zip_code,country,state,lat,lng,address
0,british,British,wl0QZqAzr1DelslQ02JGCQ,Cornish Pasty,4.5,$$,1788,Tempe,85281,US,AZ,33.423059,-111.951843,960 W University DrSte 103
1,pubs,Pubs,wl0QZqAzr1DelslQ02JGCQ,Cornish Pasty,4.5,$$,1788,Tempe,85281,US,AZ,33.423059,-111.951843,960 W University DrSte 103
2,bbq,Barbeque,Xg5qEQiB-7L6kGJ5F4K3bQ,Little Miss BBQ-University,5.0,$$,2483,Phoenix,85034,US,AZ,33.421608,-111.989151,NaN
3,newamerican,American (New),aiX_WP7NKPTdF9CfI-M-wg,Culinary Dropout,4.0,$$,2076,Tempe,85281,US,AZ,33.42853,-111.94396,NaN
4,pubs,Pubs,JzOp695tclcNCNMuBl7oxA,Four Peaks Brewing Company,4.5,$$,2585,Tempe,85281,US,AZ,33.419657,-111.915953,1340 E 8th StSte 104


In [55]:
df_yelp_business_category = df_yelp[["business_id","category"]]
df_yelp_business = df_yelp[["business_id","business_name","rating","price","lat","lng","review_count"]].drop_duplicates()
df_yelp_location = df_yelp[["lat","lng","address","city","state","country","zip_code"]].drop_duplicates()

In [56]:
df_yelp_business_category.shape

(119, 2)

In [57]:
con = sqlite3.connect("yelp_database.db")
df_yelp_business_category.to_sql(name='business_category',con=con,if_exists='replace')
df_yelp_business.to_sql(name='business',con=con,if_exists='replace')
df_yelp_location.to_sql(name='location',con=con,if_exists='replace')
cur = con.cursor()

In [58]:
# cur.execute(
# """
# SELECT * FROM business;
# """
# ).fetchall()

In [59]:
con.commit()
con.close()

In [60]:
df_yelp_business.sort_values(by='rating',ascending=False).head(10)

,business_id,business_name,rating,price,lat,lng,review_count
50,9MVKjEMN5T59uzG1xoD2BQ,Cocina Madrigal,5.0,$$,33.408799,-112.047298,2038
2,Xg5qEQiB-7L6kGJ5F4K3bQ,Little Miss BBQ-University,5.0,$$,33.421608,-111.989151,2483
62,uHZDCuzXrgWBKrGP7JVPmA,The Bread and Honey House,5.0,$,33.451287,-111.9811,318
89,2KadiQLTen9_mN_dXgKR5A,Chula Seafood,5.0,$$,33.457762,-111.90824,391
0,wl0QZqAzr1DelslQ02JGCQ,Cornish Pasty,4.5,$$,33.423059,-111.951843,1788
84,UY13CTgUqriT-fMxT2APLw,Khai Hoàn Restaurant,4.5,$,33.41434,-111.9127,557
51,2VdCHCPFzow7zcBSaPMbKA,Perfect Pear Bistro,4.5,$$,33.4215,-111.94762,404
54,d10IxZPirVJlOSpdRZJczA,Citizen Public House,4.5,$$,33.49821,-111.927963,2136
65,PNTyiqS7R-0c1ofxOfDijQ,Cafe Monarch,4.5,$$$$,33.494225,-111.931461,959
73,X1twjafYxMCpKt2MqvY5Tw,Morning Star Cafe,4.5,$,33.400259,-111.957651,414


In [61]:
df_yelp_business.to_csv("yelp_business.csv")
df_yelp_location.to_csv("yelp_location.csv")
df_yelp_business_category.to_csv("yelp_business_categories.csv")
df_categories.to_csv("yelp_categories.csv")

# Google Places API

In [62]:
google_places_key = cfg.API_KEYS["google_places"]["key"]

In [63]:
google_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params={
    'key':google_places_key,
    'location':near,
    'radius':radius,
    'keyword':'restaurant'
}
response_res = get_response(google_places_url,'',params=params)

In [64]:
JSON(response_res.json())

<IPython.core.display.JSON object>

In [65]:
google_p_restaurant_json = response_res.json()
df_g_res = pd.json_normalize(google_p_restaurant_json,
                               record_path=['results'])
df_g_res = df_g_res[["name","place_id",'geometry.location.lat', 'geometry.location.lng','rating','user_ratings_total']]
df_g_res = df_g_res.rename(columns={
    'geometry.location.lat':'lat',
    'geometry.location.lng':'lng',
})
df_g_res["keyword"] = "restaurant"
df_g_res.head()

,name,place_id,lat,lng,rating,user_ratings_total,keyword
0,Board & Brew - Tempe,ChIJAXVknywJK4cR-aIa4oTxr3I,33.417521,-111.925987,5.0,2522,restaurant
1,El Paisano Restaurant at the Rise,ChIJZSmaHdMJK4cRSFd1xrfcnNg,33.415066,-111.924187,4.7,161,restaurant
2,The Hudson Eatery & Bar,ChIJI7_nOkMJK4cRSCe7ZA1u2o0,33.414170,-111.912298,4.8,88,restaurant
3,Smile Thai Cuisine,ChIJ4zGnGJUIK4cR6lNoCN40TEM,33.405996,-111.925862,4.4,238,restaurant
4,Chou's Kitchen,ChIJUzsWau4IK4cRO6mWIAAjST8,33.415113,-111.918471,4.4,616,restaurant


In [66]:
google_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params={
    'key':google_places_key,
    'location':near,
    'radius':radius,
    'keyword':'bar'
}
response_bar = get_response(google_places_url,'',params=params)

In [67]:
google_p_bar_json = response_bar.json()
df_g_bar = pd.json_normalize(google_p_bar_json,
                               record_path=['results'])
df_g_bar = df_g_bar[["name","place_id",'geometry.location.lat', 'geometry.location.lng','rating','user_ratings_total']]
df_g_bar = df_g_bar.rename(columns={
    'geometry.location.lat':'lat',
    'geometry.location.lng':'lng',
})
df_g_bar["keyword"] = "bar"
df_g_bar.head()


,name,place_id,lat,lng,rating,user_ratings_total,keyword
0,The Hudson Eatery & Bar,ChIJI7_nOkMJK4cRSCe7ZA1u2o0,33.414170,-111.912298,4.8,88,bar
1,College,ChIJsxG0xdgIK4cR1AeYBOsn-Y8,33.422724,-111.940356,4.3,131,bar
2,Tempe Tavern,ChIJxxKaOfQIK4cRALrPs6TM94g,33.415029,-111.908546,4.3,784,bar
3,The Handlebar Tempe,ChIJO2d3KNgIK4cRvucD6Awt2m4,33.423657,-111.940256,4.0,400,bar
4,Time Out Lounge,ChIJA7wGmroIK4cR0_21Aal3jr8,33.394863,-111.938533,4.5,259,bar


In [68]:
df_g = pd.concat([df_g_res,df_g_bar])
df_g = df_g.drop_duplicates('place_id')

In [69]:
df_g.to_csv("google_places.csv")

In [70]:
con = sqlite3.connect("google_places.db")
df_g.to_sql(name='places',con=con,if_exists="replace")
con.commit()
con.close()

# Compare Places from all APIs

In [71]:
yelp_names = df_yelp_business[["business_name","rating","review_count"]]
top_yelp = yelp_names.sort_values(by=["rating"],ascending=False).head(10)
yelp_names[yelp_names["business_name"].str.contains("Four")]

,business_name,rating,review_count
4,Four Peaks Brewing Company,4.5,2585


In [72]:
df_venues_with_ratings.columns

Index(['venue_id', 'price', 'likes', 'rating', 'number_of_ratings',
       'venue_name', 'Latitude', 'Longitude', 'category_id'],
      dtype='object')

In [73]:
fs_names = df_venues_with_ratings[["venue_name","number_of_ratings","rating"]]
fs_names.sort_values(by=["rating"],ascending=False).head(10)

,venue_name,number_of_ratings,rating
2,In-N-Out Burger,53,4.444444
4,Flancer's Restaurant,59,4.444444
21,Dutch Bros. Coffee,195,4.388889
8,Four Peaks Brewing Company,683,4.388889
15,Starbucks,12,4.277778
24,In-N-Out Burger,353,4.222222
20,Lux Central,407,4.222222
16,Press Coffee,12,4.222222
6,Songbird Coffee & Tea House,93,4.166667
18,Raising Cane's Chicken Fingers,112,4.166667


In [74]:
g_names = df_g[["name","rating","user_ratings_total"]]
g_names.sort_values(by=["rating"],ascending=False).head(10)

,name,rating,user_ratings_total
0,Board & Brew - Tempe,5.0,2522
13,Pita Valley,4.9,148
2,The Hudson Eatery & Bar,4.8,88
7,Doner Dudes,4.7,564
1,El Paisano Restaurant at the Rise,4.7,161
10,Hangover Hoagies,4.7,301
12,Postino Annex,4.6,1525
18,"Snooze, an A.M. Eatery",4.6,1717
17,Haji-Baba,4.6,2160
14,Four Peaks Brewing Company,4.6,2101


In [75]:
set(yelp_names["business_name"]).intersection(set(fs_names.venue_name))

{'Four Peaks Brewing Company', 'In-N-Out Burger'}

In [76]:
set(yelp_names.business_name).intersection(set(g_names.name))

{"Casey Moore's Oyster House",
 'Four Peaks Brewing Company',
 'Haji-Baba',
 'Postino Annex',
 'Snooze, an A.M. Eatery',
 'Thai Basil'}

In [77]:
set(g_names.name).intersection(set(fs_names.venue_name))

{'El Paisano Restaurant at the Rise', 'Four Peaks Brewing Company'}

In [78]:
df_venues_with_ratings[df_venues_with_ratings.venue_name.str.contains("pai",case=False)]

,venue_id,price,likes,rating,number_of_ratings,venue_name,Latitude,Longitude,category_id
28,5db759724e10f90008b97997,$,0,NaN,NaN,El Paisano Restaurant at the Rise,33.414505,-111.924302,4bf58dd8d48988d1c4941735


In [79]:
df_yelp_business[df_yelp_business.business_name.str.contains("thai",case=False)]

,business_id,business_name,rating,price,lat,lng,review_count
95,QYIhMNwxXejDoCr-F1BxDg,Thai Basil,4.0,$$,33.421539,-111.944782,491
112,NUhHkT1wfXsTLxt_tWwVXw,Thai Elephant Bistro,4.0,$$,33.421599,-111.950366,501


In [80]:
df_g[df_g.name.str.contains("paisano",case=False)]

,name,place_id,lat,lng,rating,user_ratings_total,keyword
1,El Paisano Restaurant at the Rise,ChIJZSmaHdMJK4cRSFd1xrfcnNg,33.415066,-111.924187,4.7,161,restaurant


## Distance Matrix for top 10 from google

In [81]:
df_g.sort_values(by=['rating'],ascending=False).head(10)

,name,place_id,lat,lng,rating,user_ratings_total,keyword
0,Board & Brew - Tempe,ChIJAXVknywJK4cR-aIa4oTxr3I,33.417521,-111.925987,5.0,2522,restaurant
13,Pita Valley,ChIJURNedaEJK4cR-71mG_b4lHM,33.421455,-111.908340,4.9,148,restaurant
2,The Hudson Eatery & Bar,ChIJI7_nOkMJK4cRSCe7ZA1u2o0,33.414170,-111.912298,4.8,88,restaurant
7,Doner Dudes,ChIJlVB-cMIIK4cRvuuZ0YoUrPU,33.414122,-111.928970,4.7,564,restaurant
1,El Paisano Restaurant at the Rise,ChIJZSmaHdMJK4cRSFd1xrfcnNg,33.415066,-111.924187,4.7,161,restaurant
10,Hangover Hoagies,ChIJ76we8-4IK4cRzhP-kJHaKzA,33.415309,-111.920255,4.7,301,restaurant
12,Postino Annex,ChIJ9Wd6mGYGK4cRiWd0_bkohHg,33.423860,-111.934731,4.6,1525,bar
18,"Snooze, an A.M. Eatery",ChIJ9Wd6mGYGK4cR0SgXwes1TPI,33.423708,-111.934780,4.6,1717,restaurant
17,Haji-Baba,ChIJs4IizvMIK4cRUz-fTLAQDaE,33.414279,-111.913491,4.6,2160,restaurant
14,Four Peaks Brewing Company,ChIJY9M3BPAIK4cRkSOWsfkfkCQ,33.419705,-111.915761,4.6,2101,restaurant


In [82]:
df_g_top_10 = df_g.sort_values(by=['rating'],ascending=False).head(10)
addresses = (df_g_top_10["lat"].astype('str').str.cat(","+df_g_top_10["lng"].astype('str'))).values

In [83]:
#addresses
addresses

array(['33.4175214,-111.9259873', '33.4214553,-111.9083404',
       '33.4141705,-111.9122976', '33.4141218,-111.9289703',
       '33.4150657,-111.9241867', '33.4153086,-111.9202553',
       '33.4238601,-111.9347308', '33.4237079,-111.9347804',
       '33.4142787,-111.9134911', '33.4197052,-111.9157608'], dtype=object)

In [84]:
import urllib.request

In [85]:
# https://developers.google.com/optimization/routing/vrp#send_request
def send_request(origin_addresses, dest_addresses, API_key):
  """ Build and send request for the given origin and destination addresses."""
  def build_address_str(addresses):
    # Build a pipe-separated string of addresses
    address_str = ''
    for i in range(len(addresses) - 1):
      address_str += addresses[i] + '|'
    address_str += addresses[-1]
    return address_str

  request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial'
  origin_address_str = build_address_str(origin_addresses)
  dest_address_str = build_address_str(dest_addresses)
  request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key
  jsonResult = urllib.request.urlopen(request).read()
  response = json.loads(jsonResult)
  return response

In [86]:
# Build rows of the amtrix
def build_distance_matrix(response):
  distance_matrix = []
  for row in response['rows']:
    row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
    distance_matrix.append(row_list)
  return distance_matrix

In [87]:
def create_distance_matrix(data):
  addresses = data["addresses"]
  API_key = data["API_key"]
  # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
  max_elements = 100
  num_addresses = len(addresses) # 16 in this example.
  # Maximum number of rows that can be computed per request (6 in this example).
  max_rows = max_elements // num_addresses
  # num_addresses = q * max_rows + r (q = 2 and r = 4 in this example).
  q, r = divmod(num_addresses, max_rows)
  dest_addresses = addresses
  distance_matrix = []
  # Send q requests, returning max_rows rows per request.
  for i in range(q):
    origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  # Get the remaining remaining r rows, if necessary.
  if r > 0:
    origin_addresses = addresses[q * max_rows: q * max_rows + r]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)
  return distance_matrix

In [88]:
data = {'addresses':addresses,
       'API_key':google_places_key}

In [89]:
 distance_matrix = create_distance_matrix(data)
print(distance_matrix)

[[0, 2284, 1649, 902, 1048, 933, 1526, 1509, 1575, 1448], [2295, 0, 1200, 3171, 2214, 1872, 2804, 2787, 2277, 919], [1660, 1204, 0, 2049, 1167, 826, 3172, 3155, 167, 928], [584, 2870, 1617, 0, 1193, 901, 2112, 2096, 1543, 2034], [493, 2779, 1181, 882, 0, 465, 2021, 2004, 1107, 1943], [715, 2287, 1472, 1471, 516, 0, 2024, 2007, 1398, 1258], [1508, 2804, 3178, 2499, 2358, 2873, 0, 17, 3104, 2355], [1491, 2787, 3161, 2482, 2341, 2856, 17, 0, 3087, 2338], [1847, 1436, 167, 2236, 1400, 1058, 4032, 4015, 0, 1160], [1847, 919, 927, 2180, 1359, 1017, 2355, 2338, 1249, 0]]


In [90]:
data['distance_matrix'] = distance_matrix
data['num_vehicles'] = 1
data['depot'] = 0

In [91]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [92]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [93]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [94]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [95]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [96]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [127]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 9579 miles
Route for vehicle 0:
 0 -> 3 -> 5 -> 4 -> 8 -> 2 -> 1 -> 9 -> 7 -> 6 -> 0



In [128]:
data["addresses"]

array(['33.4175214,-111.9259873', '33.4214553,-111.9083404',
       '33.4141705,-111.9122976', '33.4141218,-111.9289703',
       '33.4150657,-111.9241867', '33.4153086,-111.9202553',
       '33.4238601,-111.9347308', '33.4237079,-111.9347804',
       '33.4142787,-111.9134911', '33.4197052,-111.9157608'], dtype=object)

In [108]:
df_g_top_10 = df_g_top_10.reset_index(drop=True)
df_g_top_10

,index,name,place_id,lat,lng,rating,user_ratings_total,keyword
0,0,Board & Brew - Tempe,ChIJAXVknywJK4cR-aIa4oTxr3I,33.417521,-111.925987,5.0,2522,restaurant
1,1,Pita Valley,ChIJURNedaEJK4cR-71mG_b4lHM,33.421455,-111.908340,4.9,148,restaurant
2,2,The Hudson Eatery & Bar,ChIJI7_nOkMJK4cRSCe7ZA1u2o0,33.414170,-111.912298,4.8,88,restaurant
3,3,Doner Dudes,ChIJlVB-cMIIK4cRvuuZ0YoUrPU,33.414122,-111.928970,4.7,564,restaurant
4,4,El Paisano Restaurant at the Rise,ChIJZSmaHdMJK4cRSFd1xrfcnNg,33.415066,-111.924187,4.7,161,restaurant
5,5,Hangover Hoagies,ChIJ76we8-4IK4cRzhP-kJHaKzA,33.415309,-111.920255,4.7,301,restaurant
6,6,Postino Annex,ChIJ9Wd6mGYGK4cRiWd0_bkohHg,33.423860,-111.934731,4.6,1525,bar
7,7,"Snooze, an A.M. Eatery",ChIJ9Wd6mGYGK4cR0SgXwes1TPI,33.423708,-111.934780,4.6,1717,restaurant
8,8,Haji-Baba,ChIJs4IizvMIK4cRUz-fTLAQDaE,33.414279,-111.913491,4.6,2160,restaurant
9,9,Four Peaks Brewing Company,ChIJY9M3BPAIK4cRkSOWsfkfkCQ,33.419705,-111.915761,4.6,2101,restaurant


In [122]:
df_g_top_10_with_routes = df_g_top_10.iloc[[0,3,5,4,8,2,1,9,7,6,0],].drop(columns='index')
df_g_top_10_with_routes = df_g_top_10_with_routes.reset_index(drop=False)
df_g_top_10_with_routes = df_g_top_10_with_routes.reset_index()

In [125]:
df_g_top_10_with_routes.loc[10,"level_0"]=0
df_g_top_10_with_routes

,level_0,index,name,place_id,lat,lng,rating,user_ratings_total,keyword
0,0,0,Board & Brew - Tempe,ChIJAXVknywJK4cR-aIa4oTxr3I,33.417521,-111.925987,5.0,2522,restaurant
1,1,3,Doner Dudes,ChIJlVB-cMIIK4cRvuuZ0YoUrPU,33.414122,-111.928970,4.7,564,restaurant
2,2,5,Hangover Hoagies,ChIJ76we8-4IK4cRzhP-kJHaKzA,33.415309,-111.920255,4.7,301,restaurant
3,3,4,El Paisano Restaurant at the Rise,ChIJZSmaHdMJK4cRSFd1xrfcnNg,33.415066,-111.924187,4.7,161,restaurant
4,4,8,Haji-Baba,ChIJs4IizvMIK4cRUz-fTLAQDaE,33.414279,-111.913491,4.6,2160,restaurant
5,5,2,The Hudson Eatery & Bar,ChIJI7_nOkMJK4cRSCe7ZA1u2o0,33.414170,-111.912298,4.8,88,restaurant
6,6,1,Pita Valley,ChIJURNedaEJK4cR-71mG_b4lHM,33.421455,-111.908340,4.9,148,restaurant
7,7,9,Four Peaks Brewing Company,ChIJY9M3BPAIK4cRkSOWsfkfkCQ,33.419705,-111.915761,4.6,2101,restaurant
8,8,7,"Snooze, an A.M. Eatery",ChIJ9Wd6mGYGK4cR0SgXwes1TPI,33.423708,-111.934780,4.6,1717,restaurant
9,9,6,Postino Annex,ChIJ9Wd6mGYGK4cRiWd0_bkohHg,33.423860,-111.934731,4.6,1525,bar


In [126]:
df_g_top_10_with_routes.to_csv("top places from google.csv")

In [131]:
distance_array = np.array(distance_matrix)
distance_array

array([[   0, 2284, 1649,  902, 1048,  933, 1526, 1509, 1575, 1448],
       [2295,    0, 1200, 3171, 2214, 1872, 2804, 2787, 2277,  919],
       [1660, 1204,    0, 2049, 1167,  826, 3172, 3155,  167,  928],
       [ 584, 2870, 1617,    0, 1193,  901, 2112, 2096, 1543, 2034],
       [ 493, 2779, 1181,  882,    0,  465, 2021, 2004, 1107, 1943],
       [ 715, 2287, 1472, 1471,  516,    0, 2024, 2007, 1398, 1258],
       [1508, 2804, 3178, 2499, 2358, 2873,    0,   17, 3104, 2355],
       [1491, 2787, 3161, 2482, 2341, 2856,   17,    0, 3087, 2338],
       [1847, 1436,  167, 2236, 1400, 1058, 4032, 4015,    0, 1160],
       [1847,  919,  927, 2180, 1359, 1017, 2355, 2338, 1249,    0]])

In [136]:
#3,5,4,8
print("Distance 1-2,2-3,3-4: "+ str(distance_array[3,5]+distance_array[5,4]+distance_array[4,8]))
#3,4,5,8
print("Distance 1-3,3-2,2-4: "+ str(distance_array[3,4]+distance_array[4,5]+distance_array[5,8]))

Distance 1-2,2-3,3-4: 2524
Distance 1-3,3-2,2-4: 3056
